In [146]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime, timedelta
import re
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import DateOffset


기본 메서드

In [147]:
def get_today(form='%Y-%m-%d'):
    mapping = {
        '%Y%m%d': datetime.now().strftime("%Y%m%d"),
        'yyyymmdd': datetime.now().strftime("%Y%m%d"),
        '%Y-%m-%d': datetime.now().strftime("%Y-%m-%d"),
        'yyyy-mm-dd': datetime.now().strftime("%Y-%m-%d"),
        'datetime': datetime.now(),
        '%Y%m%d%H': datetime.now().strftime("%Y%m%d%H"),
    }
    today = mapping[form]
    return today 

def scan_files_including_regex(file_folder, regex, option='name'):
    with os.scandir(file_folder) as files:
        lst = [file.name for file in files if re.findall(regex, file.name)]
    
    mapping = {
        'name': lst,
        'path': [os.path.join(file_folder, file_name) for file_name in lst]
    }
    return mapping[option]

def format_date(date):
    date = date.replace('-', '')
    date = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
    return date

def save_df_to_file(df, file_folder, subject, file_memo, file_code,input_date, include_index=False, file_extension='.csv', archive=False, archive_folder='./archive'):
    def get_today(form='%Y%m%d'):
        return datetime.now().strftime(form)
    try:
#         new_folder_path = os.path.join(file_folder, new_folder_name)
#         os.makedirs(new_folder_path, exist_ok=True)
        save_time = get_today()
        file_name = f'dataset-{subject}-{file_memo}-code{file_code}-date{input_date}-save{save_time}{file_extension}'
        file_path = os.path.join(file_folder, file_name)
        if os.path.exists(file_path) and archive:
            df_archive = pd.read_csv(file_path)
            os.makedirs(archive_folder, exist_ok=True)
            archive_file_name = 'archive-' + file_name
            archive_file_path = os.path.join(archive_folder, archive_file_name)
            df_archive.to_csv(archive_file_path, index=False)
            print(f'Archived: {archive_file_path}')
        df.to_csv(file_path, index=include_index, encoding='utf-8-sig')
        print(f'Saved: {file_path}')
    except Exception as e:
        print(f"Error: {e}")

In [164]:
class M8186:
    def __init__(self, fund_code, start_date =None, end_date = None, menu_code = '8186'):
        self.fund_code = fund_code
        self.menu_code = menu_code
        self.start_date = start_date
        self.end_date = end_date
        self.df = None  # 데이터프레임을 위한 초기화

        self.columns_multiindex = ['수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수', 'spx']
        self.columns_singleindex = ['수정기준가', 'KOSPI지수']

    def open_df_raw(self):
        lst = scan_files_including_regex(file_folder = './캡스톤데이터2', regex = f'menu{self.menu_code}-code{self.fund_code}')
        lst = sorted(lst, reverse = True)
        file_path = lst[0]
        full_path = os.path.join(os.getcwd(), '캡스톤데이터2', file_path)
        df = pd.read_csv(full_path)

        if df.isnull().all(axis=1).any():  # 데이터프레임에 누락된 값이 있는지 확인
            raise ValueError("데이터 파일에 누락된 부분이 존재합니다. 데이터 확인이 필요합니다.")
        return df
    
    def get_df_ref(self, columns=None):
        self.df = self.open_df_raw()
        default_columns = ['일자', '수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수']

        # 전달된 칼럼 리스트가 없으면 기본 칼럼 리스트 사용
        if columns is None:
            columns = default_columns

        # 선택된 칼럼만 데이터프레임에 적용
        self.df = self.df[columns]

        if self.start_date is None:
            self.start_date = self.df['일자'].min()

        if self.end_date is None:
            self.end_date = self.df['일자'].max()

        return self.df

    def open_df_SPX_index_raw(self):
        # 'dataset-index' 폴더에서 'dataset-price-' 패턴을 포함하는 파일 목록을 가져옴
        lst = scan_files_including_regex('./dataset-index', 'dataset-price-')
        lst = sorted(lst, reverse = True)
        file_path = lst[0]
        full_path = os.path.join(os.getcwd(), 'dataset-index', file_path)
        df = pd.read_csv(full_path)

        df['SPX INDEX'] = pd.to_numeric(df['SPX INDEX'], errors='coerce')
        df = df.dropna(subset=['SPX INDEX']).reset_index(drop=True)
        df.rename(columns={'SPX INDEX': 'spx', 'ticker': '일자'}, inplace=True)

        if df.isnull().all(axis=1).any():  # 데이터프레임에 누락된 값이 있는지 확인
            raise ValueError("데이터 파일에 누락된 부분이 존재합니다. 데이터 확인이 필요합니다.")
        return df

    def get_merged_df(self, avoid_nan = True):
        df_ref = self.get_df_ref()

        # open_df_SPX_index_raw에서 반환된 데이터프레임을 가져옴
        df_spx = self.open_df_SPX_index_raw()

        # '일자' 컬럼을 기준으로 두 데이터프레임을 병합
        # how='left' 옵션은 df_ref 데이터프레임을 기준으로 합치기 위함
        self.df = pd.merge(df_ref, df_spx, on='일자', how='left')

        # 비어 있는 값들을 각 열의 바로 앞 행의 값으로 대체
        if avoid_nan:
            self.df.fillna(method='ffill', inplace=True)

        for column in self.df.columns:
            if self.df[column].iloc[0] == 0 or pd.isna(self.df[column].iloc[0]):
                self.df.at[0, column] = self.df[column].iloc[1]
        
        return self.df

    def fill_zero_with_previous(self, columns=None):
        if columns is None:
            columns = self.columns_multiindex
        
        for column in columns:
            self.df[column] = self.df[column].replace(0, None)
            self.df[column] = self.df[column].ffill()
        return self.df

    def convert_to_float(self, columns=None):
        if columns is None:
            columns = self.columns_multiindex

        for column in columns:
            self.df[column] = self.df[column].apply(lambda x: float(x.replace(',', '' )) if isinstance (x,str) else x)
        return self.df 
            
    def filter_by_date_range(self):
        # '일자' 컬럼을 datetime 타입으로 변환
        self.df['일자'] = pd.to_datetime(self.df['일자'])

        # start_date와 end_date를 기준으로 데이터 필터링
        self.df = self.df[(self.df['일자'] >= self.start_date) & (self.df['일자'] <= self.end_date)]
        return self.df

    def calculate_cumulative_return_for_df(self, df, columns = None):
        df = df.copy()  # 명시적으로 데이터프레임 복사본 생성
        if columns is None:
            columns = self.columns_multiindex

        for column_name in columns:
            if column_name in df.columns:
                initial_value = df[column_name].iloc[0]
                updated_values = ((df[column_name] - initial_value) / initial_value) * 100
                updated_values.iloc[0] = 0  # 첫 번째 행의 수익률을 0으로 설정
                df.loc[:, column_name + ' (%)'] = updated_values
        return df

    # def get_daily_return(self):
    #     # 일일수익률을 저장할 새로운 칼럼명 지정
    #     daily_return_columns = [col + ' 일일수익률' for col in self.columns_singleindex]

    #     # 일일수익률 계산
    #     for col, daily_col in zip(self.columns_singleindex, daily_return_columns):
    #         self.df[daily_col] = ((self.df[col] - self.df[col].shift(1)) / self.df[col].shift(1)) * 100
    #         self.df.loc[self.df.index[0], daily_col] = 0  # 첫 행은 0으로 설정

    #     return self.df[daily_return_columns]

    def get_cumulative_return(self):
        df = self.calculate_cumulative_return_for_df(self.df, columns = self.columns_singleindex)
        cumulative_returns = {}

        for column_name in self.columns_singleindex:
            cumulative_return = df[column_name + ' (%)'].iloc[-1]
            cumulative_returns[column_name] = cumulative_return

        return cumulative_returns
    
    def get_annualized_return(self):
        df = self.calculate_cumulative_return_for_df(self.df, self.columns_singleindex)
        start_date = df['일자'].iloc[0]
        end_date = df['일자'].iloc[-1]
        days = (end_date - start_date).days + 1

        # 연환산 수익률 계산을 위한 딕셔너리 초기화
        annualized_returns = {}

        # days가 365일 이상일 경우에만 연환산 수익률 계산
        if days >= 365:
            for column_name in self.columns_singleindex:
                cumulative_return = df[column_name + ' (%)'].iloc[-1]
                annualized_return = (cumulative_return * 365) / days
                annualized_returns[column_name] = annualized_return
        else:
            # days가 365일 미만일 경우, 연환산 수익률을 계산하지 않음
            for column_name in self.columns_singleindex:
                annualized_returns[column_name] = None

        return annualized_returns

    def calculate_daily_returns_std(self):
        self.daily_returns = self.df[self.columns_singleindex].pct_change()
        self.daily_returns.iloc[0] = 0
        self.daily_returns_std = self.daily_returns.std()


    def get_volatility(self):
        # 변동성 계산을 위한 딕셔너리 초기화
        volatility = {}

        # daily_returns_std를 먼저 계산
        self.calculate_daily_returns_std()

        # 각 칼럼에 대한 변동성 계산
        for column_name in self.columns_singleindex:
            volatility[column_name] = self.daily_returns_std[column_name] * (365 ** 0.5) * 100

        return volatility
    
    def get_winning_ratio(self):
        # 일일 수익률 계산
        self.daily_returns = self.df[self.columns_singleindex].pct_change()
        self.daily_returns.iloc[0] = 0  # 첫 행을 0으로 설정

        # 각 칼럼에 대한 양수 수익률의 개수 계산
        positive_returns_count = (self.daily_returns > 0).sum()

        # 각 칼럼에 대한 전체 유효한 수익률의 개수 계산 (0인 값 제외)
        valid_returns_count = (self.daily_returns != 0).sum()

        # 승리 비율 계산
        winning_ratio = positive_returns_count / valid_returns_count

        return winning_ratio
        
    

    def filter_for_period(self, months):
        if months is not None:
            # 현재 가장 최근 날짜를 구함
            df_end_date = self.df['일자'].max()

            # 지정된 개월 수만큼 과거 날짜를 계산
            period_start_date = df_end_date - DateOffset(months=months)

            # period_start_date보다 이전 데이터를 필터링
            filtered_df = self.df[self.df['일자'] >= period_start_date]

            return filtered_df
        else:
            # months가 None이면 전체 데이터프레임 반환
            return self.df


    def generate_period_df(self):
        # self.df의 최대 및 최소 날짜 찾기
        df_start_date = self.df['일자'].min()
        df_end_date = self.df['일자'].max()

        # 가능한 모든 기간을 검사하여 default_periods 설정
        potential_periods = [1, 3, 6, 12, 24, 36, 48, 60]
        default_periods = []

        for period in potential_periods:
            period_start_date = df_end_date - DateOffset(months=period)
            if period_start_date >= df_start_date:
                default_periods.append(period)

        period_dfs = {}  # 각 기간에 해당하는 데이터프레임을 저장할 딕셔너리

        # 각 기간에 대한 데이터프레임 생성
        for period in default_periods:
            period_dfs[f"{period}m"] = self.filter_for_period(period)

        # YTD 데이터프레임 생성
        # 현재 연도 필터링
        current_year = pd.Timestamp.now().year
        current_year_df = self.df[self.df['일자'].dt.year == current_year]

        # 현재 연도 데이터가 1월 1일부터 시작하는지 확인
        if current_year_df['일자'].min() == pd.Timestamp(year=current_year, month=1, day=1):
            period_dfs['YTD'] = current_year_df

        return period_dfs

    def format_period(self, period):
        """
        '기간' 값을 포맷하는 함수. 
        예: '1m' -> '1개월', '12m' -> '1년' 등
        """
        try:
            months = int(period.replace('m', ''))
            if months < 12:
                return f'{months}개월'
            elif months % 12 == 0:
                years = months // 12
                return f'{years}년'
        except ValueError:
            return period  # 만약 다른 형식이라면 원본 값을 반환

    def get_final_cumulative_returns(self, period_dfs):
        final_returns_data = []

        for period, df in period_dfs.items():
            formatted_period = self.format_period(period)  # 기간 포맷 변경
            last_row = df.iloc[-1]
            row_data = {
                '기간': formatted_period,
                '펀드': last_row.get('수정기준가 (%)', None),
                'KOSPI': last_row.get('KOSPI지수 (%)', None),
                'KOSPI200': last_row.get('KOSPI200지수 (%)', None),
                'KOSDAQ': last_row.get('KOSDAQ지수 (%)', None),
                'S&P 500': last_row.get('spx (%)', None)
            }
            final_returns_data.append(row_data)

        # 데이터를 기반으로 새로운 데이터프레임 생성
        final_returns_df = pd.DataFrame(final_returns_data)
        final_returns_df.set_index('기간', inplace=True)

        return final_returns_df
    
    def process_period_dfs(self):
        # 각 기간별 데이터프레임을 생성
        period_dfs = self.generate_period_df()

        # 각 데이터프레임에 대해 누적 수익률 계산
        for period, df in period_dfs.items():
            period_dfs[period] = self.calculate_cumulative_return_for_df(df)

        # 전체 기간에 대한 누적수익률 추가
        period_dfs['설정이후'] = self.calculate_cumulative_return_for_df(self.df)
        
        # 각 기간별 누적수익률의 마지막 값으로 구성된 데이터프레임을 반환
        final_returns_df = self.get_final_cumulative_returns(period_dfs)

        return final_returns_df


    #기간별 수익률을 위한 메인 메서드 
    def period_cumulative_return(self):
        self.get_merged_df()
        self.filter_by_date_range()
        self.convert_to_float()
        self.fill_zero_with_previous()
        #convert_to_float 이 먼저 실행되고 fill_zero_with_previous 가 실행되어야 함 
        final_returns_df = self.process_period_dfs()

        return final_returns_df

    #성능평가지표를 위한 메인 메서드
    def investment_performance(self):
        self.get_df_ref(['일자']+self.columns_singleindex)
        self.filter_by_date_range()
        self.convert_to_float(self.columns_singleindex)
        self.fill_zero_with_previous(self.columns_singleindex)
        self.calculate_daily_returns_std()
        df = self.calculate_cumulative_return_for_df(self.df, self.columns_singleindex)

        return df
        


보고서와 데이터와의 기간 차이로 인해 오차 발생

보고서의 기간은 2021.07.29 ~ 2023.10.31

데이터의 기간은 2021.07.29 ~ 2023.10.30

In [149]:
m = M8186(fund_code = '100004')
m.period_cumulative_return()


C:\Users\hanil\AppData\Local\Temp\ipykernel_6060\461909072.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df.fillna(method='ffill', inplace=True)


,펀드,KOSPI,KOSPI200,KOSDAQ,S&P 500
기간,,,,,
1개월,-5.320985,-6.268382,-5.436014,-9.975982,-2.827159
3개월,-5.550120,-11.416161,-10.196785,-17.140543,-9.065673
6개월,3.902052,-7.634528,-5.363597,-10.169311,-0.063797
1년,36.941799,1.858138,4.707517,10.105725,6.812507
2년,30.043045,-22.221512,-20.470049,-23.702800,-9.522776
YTD,22.904183,3.315596,6.131913,11.457551,8.525068
설정이후,28.396716,-28.745008,-28.065846,-27.487956,-5.709922


In [150]:
m.open_df_SPX_index_raw()

,일자,spx
0,2021-01-04,3700.65
1,2021-01-05,3726.86
2,2021-01-06,3748.14
3,2021-01-07,3803.79
4,2021-01-08,3824.68
...,...,...
723,2023-11-16,4508.24
724,2023-11-17,4514.02
725,2023-11-20,4547.38
726,2023-11-21,4538.19


In [165]:
m = M8186(fund_code = '100004')
m.investment_performance()

,일자,수정기준가,KOSPI지수,수정기준가 (%),KOSPI지수 (%)
0,2021-07-29,1000.01,3242.65,0.000000,0.000000
1,2021-07-30,1000.02,3202.32,0.001000,-1.243736
2,2021-07-31,1000.02,3202.32,0.001000,-1.243736
3,2021-08-01,1000.03,3202.32,0.002000,-1.243736
4,2021-08-02,1000.01,3223.04,0.000000,-0.604752
...,...,...,...,...,...
819,2023-10-26,1271.60,2299.08,27.158728,-29.098731
820,2023-10-27,1280.08,2302.81,28.006720,-28.983702
821,2023-10-28,1280.12,2302.81,28.010720,-28.983702
822,2023-10-29,1280.16,2302.81,28.014720,-28.983702


In [152]:
m.get_cumulative_return()

{'수정기준가': 28.396716032839674, 'KOSPI지수': -28.745007941035876}

In [153]:
m.get_annualized_return()

{'수정기준가': 12.578642417459323, 'KOSPI지수': -12.73292220689089}

In [154]:
m.get_volatility()

{'수정기준가': 19.19123518240374, 'KOSPI지수': 16.153626441399805}

In [155]:
m.daily_returns_std

수정기준가      0.010045
KOSPI지수    0.008455
dtype: float64

In [156]:
m1 = m.daily_returns_std
m1 * (825**0.5) *100

수정기준가      28.852501
KOSPI지수    24.285697
dtype: float64

In [157]:
m.daily_returns

,수정기준가,KOSPI지수
0,0.000000,0.000000
1,0.000010,-0.012437
2,0.000000,0.000000
3,0.000010,0.000000
4,-0.000020,0.006470
...,...,...
819,-0.012518,-0.027120
820,0.006669,0.001622
821,0.000031,0.000000
822,0.000031,0.000000


In [166]:
m.get_winning_ratio()

수정기준가      0.685680
KOSPI지수    0.661408
dtype: float64